In [ ]:
for i in [1,2,3]:
    print(i)

## Iterators and Generators in Python 

Recall that **for** loops in python iterate over objects of various kinds: lists, dictionaries, sets, map objects, ... 

One way to make your favourite class also have this feature is to make it satisfy the **iterator** protocol. That is, 
it provides a function **\_\_iter\_\_** that returns a iterator class. 

An **iterator class** must implement an a method called **\_\_next\_\_** (as well as **\_\_iter\_\_** again!). The idea is that the repeatedly calling **\_\_next\_\_** will carry out the iteration. 

(The need for **\_\_iter\_\_** is perverse, it is expected to return the self and is needed to ensure asking for an iterator from an iterator returns the same iterator. Ugh)


In [ ]:
class IWantToIterate:
    
    def __iter__(self):
        return(self)
    
    def __next__(self):
        return(1)

In [ ]:
x = IWantToIterate()
print(next(x))
print(next(x))

In [ ]:
for e in x:
    print(e)

So how does an **iterator** tell the for loop to stop? i.e. indicate that it has reached the end of iteration.
This is done by throwing a **StopIteration** exception.

In [ ]:
class AStoppingIterator:
    
    def __iter__(self):
        self.count = 0
        return(self)
    
    def __next__(self):
        if self.count >= 10:
            raise StopIteration
        self.count = self.count + 1
        return(self.count-1)
        

In [ ]:
x = AStoppingIterator()
next(x)

No call to **\_\_iter\_\_** means **self.count** is undefined. In a **for** loop there is an implicit call to get the iterator.

In [ ]:
for e in x:
    print(e)

You can see the exception by omitting the for loop and calling next sufficient number of times.

In [ ]:
x = iter(AStoppingIterator())
next(x)
next(x)
next(x)
next(x)
next(x)
next(x)
next(x)
next(x)
next(x)
next(x)
next(x)
next(x)


And just to prove the point that to iterate over a class it only needs to return an **iterator** and need not provide **\_\_next\_\_**

In [ ]:
class LookMaNoNext:
    
    def __iter__(self):
        return AStoppingIterator()

class AStoppingIterator:
    
    def __init__(self):
        self.count = 0
    
    def __iter__(self):
        return(self)
    
    def __next__(self):
        if self.count >= 10:
            raise StopIteration
        self.count = self.count + 1
        return(self.count-1)
        

In [ ]:
x = LookMaNoNext()
for e in x:
    print(e)

### Generators

A generator is a function that uses **yield**. The command **yield** is like **return** but it freezes the function execution right here so that the next call to the function continues from this point. These act as iterators, returning the yielded value.  A **return** from such a function raises the **StopIteration** exception.

In [ ]:
class AStoppingIterator:
    
    def mygenerator(self):
        count = 0
        while True:
            if count >= 10:
                return
            count = count + 1
            yield(count - 1)

In [ ]:
x = AStoppingIterator()
for e in x.mygenerator():
    print(e)

In [ ]:
class Test:
    
    def fn(self):
        x = 5
        yield(x)
        print(1)
        x = x+1
        yield(x)

In [ ]:
x = Test()
for y in x.fn():
    print(y)
    print("---------")

### A python class for Graphs 

Let's begin with a adjacency matrix representation. We would like this detail to be hidden away and the user of this class should be able to use it transparently without concern about the underlying representation.

As we have seen in our reachability and connected components algorithms, we will need the ability to 
       * iterate over vertices
       * iterate over neighbours of a vertex.
       
We consider the iteration over vertices first and then iteration over neighbours.


In [ ]:
import numpy as np
class Graph:
    
    def __init__(self,n,E=[]):
        self.N = n
        self.AdjMat = np.zeros([self.N,self.N],dtype=np.int32)
        for e in E:
            x,y = e
            self.AdjMat[x,y]=1
            self.AdjMat[y,x]=1
            
    def __str__(self):
        return str(self.AdjMat)
        
    def __iter__(self):
        self.index = 0
        return(self)
        
    def __next__(self):
        if self.index >= self.N:
            raise StopIteration
        rval = self.index
        self.index = self.index+1
        return rval
    
            

In [ ]:
G = Graph(6,[(1,2),(2,3),(3,4),(4,1),(5,0)])

In [ ]:
for u in G:
    print(u)

In [ ]:
for u in G:
    print(u)
for v in G:
    print(v)

In [ ]:
for u in G:
    for v in G:
        print(u,v)

So, that is not nice, but expected. The iterator has only one "state" and so if you want two of them in parallel they share their state.

If you have two graphs instead of one then nested loops work. Each has its own state.

In [ ]:
H = Graph(3,[(0,1)])

In [ ]:
for e in G:
    for f in H:
        print(e,f)

Switching to generators won't solve it either.

In [ ]:
import numpy as np
class Graph:
    
    def __init__(self,n,E=[]):
        self.N = n
        self.AdjMat = np.zeros([self.N,self.N],dtype=np.int32)
        for e in E:
            x,y = e
            self.AdjMat[x,y]=1
            self.AdjMat[y,x]=1
            
    def __str__(self):
        return str(self.AdjMat)
        
    def vertices(self):
        self.index = 0
        while self.index < self.N:
            yield self.index
            self.index = self.index+1
    
            

In [ ]:
G = Graph(6,[(1,2),(2,3),(3,4),(4,1),(5,0)])
for v in G.vertices():
    print(v)

In [ ]:
for v in G.vertices():
    for u in G.vertices():
        print(v,u)

This problem can be fixed by ensuring each iterator has its own state. 

So, we create a new object with the iterator so that it an keep this state.

In [ ]:
import numpy as np
class Graph:
    
    def __init__(self,n,E=[]):
        self.N = n
        self.AdjMat = np.zeros([self.N,self.N],dtype=np.int32)
        for e in E:
            x,y = e
            self.AdjMat[x,y]=1
            self.AdjMat[y,x]=1
            
    def __str__(self):
        return str(self.AdjMat)
    
    def __iter__(self):
        return self.Vertices(self.N)
    
    class Vertices:
        
        def __init__(self,nu):
            self.Nu = nu
            self.index = 0
        
        def __iter__(self):
            return(self)
        
        def __next__(self):
            if self.index >= self.Nu:
                raise StopIteration
            rval = self.index
            self.index = self.index+1
            return rval
            

in above code we kept vertices class inside graph but to just show that it can be kept out also we have written the below code nothing much significant change occurs the logic remains same

In [1]:
import numpy as np
class Graph:
    
    def __init__(self,n,E=[]):
        self.N = n
        self.AdjMat = np.zeros([self.N,self.N],dtype=np.int32)
        for e in E:
            x,y = e
            self.AdjMat[x,y]=1
            self.AdjMat[y,x]=1
            
    def __str__(self):
        return str(self.AdjMat)
    
    def __iter__(self):
        return Vertices(self.N)
    
class Vertices:
        
    def __init__(self,nu):
        self.Nu = nu
        self.index = 0
        
    def __iter__(self):
        return(self)
        
    def __next__(self):
        if self.index >= self.Nu:
            raise StopIteration
        rval = self.index
        self.index = self.index+1
        return rval
            

In [2]:
G = Graph(6,[(1,2),(2,3),(3,4),(4,1),(5,0)])

In [3]:
for i in G:
    for j in G:
        print(i,j)

0 0
0 1
0 2
0 3
0 4
0 5
1 0
1 1
1 2
1 3
1 4
1 5
2 0
2 1
2 2
2 3
2 4
2 5
3 0
3 1
3 2
3 3
3 4
3 5
4 0
4 1
4 2
4 3
4 4
4 5
5 0
5 1
5 2
5 3
5 4
5 5
